### seq2seq 모형 학습

In [1]:
# 랜덤 시드 설정
import numpy as np
import tensorflow as tf
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
# 모형 변수 설정
n_batch = 64                            # 배치 크기
epochs = 100                            # 학습 epochs
latent_dim = 256                        # 단어 인코딩 축소 차원
n_max_sample = 10000                    # 학습 시킬 최대 샘플 수
data_path = './data/seq2seq/fra.txt'    # 데이터 텍스트 파일 경로

In [3]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
lines[:10]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)',
 'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba

In [5]:
# 인풋, 타깃 텍스트 데이터 정리
x_txts = [] # 영어 문장 모음
y_txts = [] # 한글 문장 모음
x_chars_uni = set() # 영어 문장 토큰 모음
y_chars_uni = set() # 한글 문장 토큰 모음
n_sample = min(n_max_sample, len(lines)-1)

for line in lines[:n_sample]:
    x_txt, y_txt, _ = line.split('\t')
    y_txt = '\t' + y_txt + '\n'
    x_txts.append(x_txt)
    y_txts.append(y_txt)

    for char in x_txt:
        if char not in x_chars_uni:
            x_chars_uni.add(char)
    for char in y_txt:
        if char not in y_chars_uni:
            y_chars_uni.add(char)

In [6]:
# 토큰 정리
x_chars_uni = sorted(list(x_chars_uni))
y_chars_uni = sorted(list(y_chars_uni))
n_encoder_tokens = len(x_chars_uni)
n_decoder_tokens = len(y_chars_uni)
print('유니크 인코더 토큰 글자 수: %d'%n_encoder_tokens)
print('유니크 디코더 토큰 글자 수: %d'%n_decoder_tokens)

max_encoder_seq_len = 0
for txt in x_txts:
    txt_len = len(txt)
    max_encoder_seq_len = max(txt_len,
                              max_encoder_seq_len)
print('인코더 문장 내 최대 문자 수: %d'%max_encoder_seq_len)

max_decoder_seq_len = 0
for txt in y_txts:
    txt_len = len(txt)
    max_decoder_seq_len = max(txt_len,
                              max_decoder_seq_len)
print('디코더 문장 내 최대 문자 수: %d'%max_decoder_seq_len)

유니크 인코더 토큰 글자 수: 71
유니크 디코더 토큰 글자 수: 93
인코더 문장 내 최대 문자 수: 15
디코더 문장 내 최대 문자 수: 59


In [7]:
# 토큰 인덱스
x_token_idx = {}
for idx, char in enumerate(x_chars_uni):
    x_token_idx[char] = idx

y_token_idx = {}
for idx, char in enumerate(y_chars_uni):
    y_token_idx[char] = idx

In [8]:
# 영 행렬 만들기
encoder_x_data = np.zeros((len(x_txts),
                             max_encoder_seq_len,
                             n_encoder_tokens),
                          dtype='float32')
decoder_x_data = np.zeros((len(x_txts),
                             max_decoder_seq_len,
                             n_decoder_tokens),
                          dtype='float32')
decoder_y_data = np.zeros((len(x_txts),
                             max_decoder_seq_len,
                             n_decoder_tokens),
                          dtype='float32')

In [9]:
# 인풋 데이터 행렬
for i, x_txt in enumerate(x_txts):
    for t, char in enumerate(x_txt):
        encoder_x_data[i, t, x_token_idx[char]] = 1.
    encoder_x_data[i, t + 1:, x_token_idx[' ']] = 1.

In [10]:
# 타겟 데이터 행렬
for i, y_txt in enumerate(y_txts):
    for t, char in enumerate(y_txt):
        decoder_x_data[i, t, y_token_idx[char]] = 1.
        if t > 0:
            decoder_y_data[i, t - 1, y_token_idx[char]] = 1.

    decoder_x_data[i, t + 1:, y_token_idx[' ']] = 1.
    decoder_y_data[i, t:, y_token_idx[' ']] = 1.

In [11]:
# 인코더 모형 생성
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed

encoder_inputs = Input(shape=(None, n_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [12]:
# 디코더 모형 생성
decoder_inputs = Input(shape=(None, n_decoder_tokens))
decoder = LSTM(latent_dim,
                return_sequences=True,
                return_state=True)
decoder_outs, _, _ = decoder(decoder_inputs,
                             initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(n_decoder_tokens,
                                      activation='softmax'))
decoder_outputs = decoder_dense(decoder_outs)

In [13]:
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  358400      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [14]:
# 모델 컴파일
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
# 학습
model.fit([encoder_x_data, decoder_x_data], decoder_y_data,
          batch_size=n_batch,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 20s 3ms/sample - loss: 1.1566 - accuracy: 0.7355 - val_loss: 1.0397 - val_accuracy: 0.7086
Epoch 2/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.8282 - accuracy: 0.7755 - val_loss: 0.8350 - val_accuracy: 0.7732
Epoch 3/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.6622 - accuracy: 0.8139 - val_loss: 0.7048 - val_accuracy: 0.7978
Epoch 4/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.5762 - accuracy: 0.8322 - val_loss: 0.6368 - val_accuracy: 0.8158
Epoch 5/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.5273 - accuracy: 0.8457 - val_loss: 0.5961 - val_accuracy: 0.8257
Epoch 6/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.4925 - accuracy: 0.8553 - val_loss: 0.5587 - val_accuracy: 0.8377
Epoch 7/100
8000/8000 [==============================] - 19s 2ms

In [16]:
# 추론 모형 생성
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,
                         decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [17]:
# 토큰 리버스 인덱스
reverse_x_char_idx = {}
for char, idx in x_token_idx.items():
    reverse_x_char_idx[idx] = char

reverse_y_char_idx = {}
for char, idx in y_token_idx.items():
    reverse_y_char_idx[idx] = char

In [18]:
# 결과값 디코딩
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    y_seq = np.zeros((1,1,n_decoder_tokens))
    y_seq[0, 0, y_token_idx['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [y_seq] + states_value)

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        sampled_char = reverse_y_char_idx[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
                len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        y_seq = np.zeros((1,1,n_decoder_tokens))
        y_seq[0, 0, sampled_token_index] = 1.

        states_value = [h,c]
    return decoded_sentence

In [19]:
for seq_idx in range(100):
    x_seq = encoder_x_data[seq_idx:seq_idx + 1]
    decoded_sentence = decode_sequence(x_seq)
    print('-')
    print('Input sentence:', x_txts[seq_idx])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Inp